In [1]:
!pip install transformers
!pip install eli5
!pip install scipy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 64.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 44.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.8/199.8 KB 7.0 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 216.2/216.2 KB 12.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.1/133.1 KB 15.3 MB/s eta 0:00:00
  Created wheel for eli5: filename=eli5-0.13.0-py2.py3-none-any.whl size=107748 sha256=bce20ef3d68f11d20f739306ae011849d4e3114a681f93eedb2bf6ebd0a25353
  Stored in directory: /root/.cache/pip/wheels/85/ac/25/ffcd87ef8f9b1eec324fdf339359be71f22612459d8c75d89c
Successfully built eli5
  Attempting uninstall: jinja2
    Found existing installation: Jinja2 2.11.3

In [3]:
import scipy
import numpy as np
from scipy import stats

#error =  cannot import itemfreq from scipyl.stats
# below code solved the issue 
def monkeypath_itemfreq(sampler_indices):
   return zip(*np.unique(sampler_indices, return_counts=True))
scipy.stats.itemfreq=monkeypath_itemfreq

import eli5
from eli5.lime import TextExplainer
from typing import List
from eli5.lime.samplers import MaskingTextSampler, MaskingTextSamplers
from transformers import AutoTokenizer
from IPython.core.display import HTML, display


import torch 
device =torch.device('cuda')

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
model=torch.load(r'/content/drive/MyDrive/Saved Models/MuRIL.pt',map_location=torch.device('cuda'))

In [6]:
tokenizer = AutoTokenizer.from_pretrained('google/muril-base-cased', do_lower_case=True)

In [25]:
def adapter(data):
  #inputs = tokenizer(data, return_tensors="pt")
  inputs = tokenizer(data, 
          return_tensors='pt', 
          padding=True, 
          truncation=True, 
          max_length=128)

  with torch.no_grad():
      #m = model(**inputs)
      #print(model(**inputs).logits)
      logits = model(**inputs).logits
      scores = model(**inputs)[0].softmax(1).detach().cpu().numpy()

  #predicted_class_id = logits.argmax().item()
  # classifications = ['Introduction','Depression','Grey Area']
  # classification = logits.argmax().item()
  # class_name = classifications[classification]
  
  return scores


In [26]:
text = 'मैं न्यूयॉर्क में अपने दादा-दादी के साथ पैदा हुई और पली-बढ़ी'

In [27]:
te = TextExplainer( random_state=42,sampler=MaskingTextSampler())

In [28]:
# Show the explanation with the actual words
te.fit(text, adapter)
te.explain_prediction(target_names=['Introduction','Depression','Grey Area'])

/usr/local/lib/python3.8/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


Explanation(estimator="SGDClassifier(alpha=0.001, loss='log', penalty='elasticnet',\n              random_state=RandomState(MT19937) at 0x7F4A919CD840)", description=None, error=None, method='linear model', is_regression=False, targets=[TargetExplanation(target='Introduction', feature_weights=FeatureWeights(pos=[FeatureWeight(feature='म', weight=1.4750816157252773, std=None, value=2.0), FeatureWeight(feature='द', weight=0.9778675455459933, std=None, value=5.0), FeatureWeight(feature='क स', weight=0.3269641899714533, std=None, value=1.0), FeatureWeight(feature='प द', weight=0.3174432327457605, std=None, value=1.0), FeatureWeight(feature='स थ', weight=0.2812863179042277, std=None, value=1.0), FeatureWeight(feature='द ह', weight=0.24552869174490732, std=None, value=1.0), FeatureWeight(feature='र क', weight=0.23431413431473155, std=None, value=1.0), FeatureWeight(feature='पल बढ', weight=0.2259008796251581, std=None, value=1.0), FeatureWeight(feature='न य', weight=0.17401847269388201, std=None, value=1.0), FeatureWeight(feature='ह', weight=0.13947326461164392, std=None, value=1.0), FeatureWeight(feature='थ प', weight=0.13711471662183444, std=None, value=1.0), FeatureWeight(feature='और पल', weight=0.13068282864527736, std=None, value=1.0), FeatureWeight(feature='अपन द', weight=0.1256644952500583, std=None, value=1.0), FeatureWeight(feature='न', weight=0.09393997180602673, std=None, value=1.0), FeatureWeight(feature='ई और', weight=0.08872524184263923, std=None, value=1.0), FeatureWeight(feature='म अपन', weight=0.08292667829131647, std=None, value=1.0), FeatureWeight(feature='बढ', weight=0.06173747356710216, std=None, value=1.0), FeatureWeight(feature='और', weight=0.05327677530982404, std=None, value=1.0), FeatureWeight(feature='द क', weight=0.044765443784915884, std=None, value=1.0), FeatureWeight(feature='य र', weight=0.0055725341417462385, std=None, value=1.0)], neg=[FeatureWeight(feature='<BIAS>', weight=-0.9422177874422996, std=None, value=1.0), FeatureWeight(feature='द द', weight=-0.6535379726570729, std=None, value=3.0), FeatureWeight(feature='पल', weight=-0.32897124402592287, std=None, value=1.0), FeatureWeight(feature='स', weight=-0.320048191955499, std=None, value=1.0), FeatureWeight(feature='थ', weight=-0.3164690829992183, std=None, value=1.0), FeatureWeight(feature='अपन', weight=-0.24675222174810216, std=None, value=1.0), FeatureWeight(feature='प', weight=-0.24355375986379302, std=None, value=1.0), FeatureWeight(feature='म न', weight=-0.17865056328387613, std=None, value=1.0), FeatureWeight(feature='ई', weight=-0.1447117597198108, std=None, value=1.0), FeatureWeight(feature='र', weight=-0.11404489534124149, std=None, value=1.0), FeatureWeight(feature='ह ई', weight=-0.08525581834688303, std=None, value=1.0), FeatureWeight(feature='क', weight=-0.08318555125368392, std=None, value=2.0)], pos_remaining=0, neg_remaining=0), proba=0.79883277241218, score=1.5648856555063726, weighted_spans=WeightedSpans(docs_weighted_spans=[DocWeightedSpans(document='मैं न्यूयॉर्क में अपने दादा-दादी के साथ पैदा हुई और पली-बढ़ी', spans=[('म', [(0, 1)], 1.4750816157252773), ('न', [(4, 5)], 0.09393997180602673), ('र', [(10, 11)], -0.11404489534124149), ('क', [(12, 13)], -0.08318555125368392), ('म', [(14, 15)], 1.4750816157252773), ('अपन', [(18, 21)], -0.24675222174810216), ('द', [(23, 24)], 0.9778675455459933), ('द', [(25, 26)], 0.9778675455459933), ('द', [(28, 29)], 0.9778675455459933), ('द', [(30, 31)], 0.9778675455459933), ('क', [(33, 34)], -0.08318555125368392), ('स', [(36, 37)], -0.320048191955499), ('थ', [(38, 39)], -0.3164690829992183), ('प', [(40, 41)], -0.24355375986379302), ('द', [(42, 43)], 0.9778675455459933), ('ह', [(45, 46)], 0.13947326461164392), ('ई', [(47, 48)], -0.1447117597198108), ('और', [(49, 51)], 0.05327677530982404), ('पल', [(52, 54)], -0.32897124402592287), ('बढ', [(56, 58)], 0.06173747356710216), ('म न', [(0, 1), (4, 5)], -0.17865056328387613), ('न य', [(4, 5), (6, 7)], 0.17401847269388201), ('य र', [(8, 9), (10, 11)], 0.00